In [ ]:
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla
from scipy.constants import speed_of_light, elementary_charge, electron_mass, hbar as hbar_SI
from scipy.sparse.linalg import eigsh
import matplotlib.pyplot as plt

In [ ]:
me_SI = electron_mass
e_SI = elementary_charge
c_SI = speed_of_light

meV = e_SI * 1e-3
nm = 1e-9
ps = 1e-12

c = c_SI * ps / nm
hbar = hbar_SI / (meV * ps)
m = me_SI * c_SI ** 2 / meV / c ** 2

In [ ]:
x_min, x_max = -50, 150
t_min, t_max = 0, 125
Nx, Nt = 5000, 5000

In [ ]:
x_values = np.linspace(x_min, x_max, Nx)
t_values = np.linspace(t_min, t_max, Nt)

dx = x_values[1] - x_values[0]
dt = t_values[1] - t_values[0]

In [ ]:
print(f"dx = {dx}")
print(f"dt = {dt}")

In [ ]:
m = m * 0.98 
omega = 1 / hbar												# from matlab reference pdf in box
alpha_ent_barr = 0.4900									# from 2019Yamahata paper			#		(meV / V)
alpha_ent_exit_barr = 0.0370						# paper												#   (meV / V)
alpha_exit_barr = 0.4800								# paper												#		(meV / V)
alpha_exit_ent_bar = 0.0520							# paper												#		(meV / V)
V_ent = -0.7000		#V										# paper
V_exit = -0.7000	#V										# paper
V_amp = 1.4150		#V?										# matlab reference
f = 4																		# matlab reference
x_ent = 0		#nm													# paper
x_exit = 100	#nm												# paper
U_scr = 1		#meV												# paper
L_ent = 100	#nm													# matlab reference
L_exit = 100 #nm												# matlab reference
L_scr = 1 #nm														# paper

In [ ]:
def theta(x):
  if isinstance(x, int) or isinstance(x, float):
    return 1 if x > 0 else 0
  return np.where(x > 0, 1, 0)

In [ ]:
def V_ac(t):
  return V_amp * np.cos(2 * np.pi * f * t * 1e-3)

In [ ]:
def U_ent(x, t):
  A = -alpha_ent_barr * (V_ent + V_ac(t))
  B = (alpha_ent_barr / alpha_ent_exit_barr) ** (-np.abs(x - x_ent) / np.abs(x_exit - x_ent))
  return A * B

In [ ]:
def U_exit(x):
  A = -alpha_exit_barr * V_exit
  B = (alpha_exit_barr / alpha_exit_ent_bar) ** (-np.abs(x - x_exit) / np.abs(x_exit - x_ent))
  return A * B

In [ ]:
def U_upper(x):
  A = U_scr * np.exp(-((x - x_ent) / L_scr) * theta(x - x_ent)) * np.exp(-((x_ent - L_ent - x) / L_scr) * theta(x_ent - L_ent - x))
  B = U_scr * np.exp(-((x - x_exit - L_exit) / L_scr) * theta(x - x_exit - L_exit)) * np.exp(-((x_exit - x) / L_scr) * theta(x_exit - x))
  return A + B

In [ ]:
def U(x, t):
  return U_ent(x, t) + U_exit(x) + U_upper(x)

In [ ]:
V = U(x_values, 0)

In [ ]:
U(150, 0)

In [ ]:
plt.plot(x_values, V)

In [ ]:
# for t in [0, 20, 40, 80, 125]:
#   V = U(x_values, t)
#   plt.plot(x_values, V)
# plt.title("Potential U(x, t) vs x")
# plt.xlabel("x")
# plt.ylabel("Potential U(x, t)")
# plt.show()

In [ ]:
laplacian = sp.diags([1, -2, 1], offsets=[-1, 0, 1], shape=(Nx, Nx), format='csc') / dx ** 2
H_kinetic = - (hbar ** 2 / (2 * m)) * laplacian

In [ ]:
H = H_kinetic + sp.diags(V, format="csc")

In [ ]:
w, v = eigsh(H, k=1, which="SA")

In [ ]:
w

In [ ]:
psi0_in = v[:, 0]

In [ ]:
psi0 = psi0_in

In [ ]:
psi0 /= np.sqrt(np.sum(np.abs(psi0) ** 2) * dx)

In [ ]:
residual = H * psi0 - w[0] * psi0
residual_norm = np.linalg.norm(residual)
vector_norm = np.linalg.norm(psi0)
print(residual_norm / vector_norm)

In [ ]:
plt.plot(x_values, psi0)

In [ ]:
psi_real = np.zeros((Nx, Nt))
psi_img = np.zeros((Nx, Nt))

psi_real[:, 0] = np.real(psi0)
psi_img[:, 0] = np.imag(psi0)

In [ ]:
I = sp.eye(Nt, format="csc")

In [ ]:
psi = psi0

In [ ]:
for t_i in range(1, Nt):
	if t_i % 100 == 0:
		print(f"Step {t_i} / {Nt}")	

	# t_in = (t_min + (t_i - 1) * dt)
	U_arr = U(x_values, t_values[t_i])
	# U_min = np.argmin(U_arr)
	# qd_min = x_min + U_min * dx
	# V = 1 / 2 * m * omega ** 2 * (x_values - qd_min) ** 2
	H = H_kinetic + sp.diags(U_arr, 0, format='csc')
 
	

	A = I + 1j * dt / (2 * hbar) * H
	B = I - 1j * dt / (2 * hbar) * H

	psi = spla.spsolve(A, B @ psi)
	norm = np.sqrt(np.sum(np.abs(psi) ** 2) * dx)

	if norm > 0:
		psi /= norm
	else:
		print(f"Zero norm at step {t_i}")
		break

	psi_real[:, t_i] = np.real(psi)
	psi_img[:, t_i] = np.imag(psi)

In [ ]:
plt.figure(figsize=(8, 6), dpi=200)
psi_sq = psi_real ** 2 + psi_img ** 2

plt.pcolormesh(x_values, t_values, psi_sq.transpose(), shading="auto")
plt.colorbar(label='|psi(x,t)|^2')
plt.xlabel('x')
plt.ylabel('t')
plt.title(f'Numerical |psi|^2')
plt.show()

In [ ]:
from scipy.integrate import simpson

norms = []

for i in range(5000):
    psi_sq = psi_real[:, i] ** 2 + psi_img[:, i] ** 2
    normalization = simpson(psi_sq, x=x_values)
    norms.append(np.round(normalization, 4))

plt.figure(figsize=(8,5))
plt.plot(t_values, norms, '-', label='Normalization')
plt.axhline(1.0, color='red', linestyle='--', label='Expected = 1')
plt.xlabel('Time (ps)')
plt.ylabel('Normalization')
plt.title('Wavefunction Normalization Over Time')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.plot(x_values, psi_real[:, 0])

In [ ]:
x_values = np.linspace(-50, 150, 1000)
dx = x_values[1] - x_values[0]
V = U(x_values, 0)
laplacian = sp.diags([1, -2, 1], offsets=[-1, 0, 1], shape=(1000, 1000), format='csc') / dx ** 2
H_kinetic = - (hbar ** 2 / (2 * m)) * laplacian
H = H_kinetic + sp.diags(V, format="csc")
w, v = eigsh(H, k=1, which="SA")
psi0 = v[:, 0]
psi0 /= np.sqrt(np.sum(np.abs(psi0) ** 2) * dx)

In [ ]:
plt.plot(x_values, psi0)

In [ ]:
residual = H * psi0 - v[:, 0] * psi0
residual_norm = np.linalg.norm(residual)
vector_norm = np.linalg.norm(psi0)
print(residual_norm / vector_norm)